In [0]:
import numpy as np

In [0]:
l1 = dbutils.fs.ls('dbfs:/FileStore/demo-files')
len(l1)

Out[35]: 4

In [0]:
from pyspark.sql.functions import * 

In [0]:
df_main = spark.createDataFrame([], schema=StructType([]))

In [0]:
first_time = True
for file in l1:
    print(f'\nFile Path :- {file.path}')
    df = spark.read.csv(file.path, header=True)
#     arr = [data[0]for data in df.select('address').collect()]
    df_bool = df.withColumn('address', col('address').rlike('[a-zA-Z][*!@^&?%#$]').alias('address'))
    df_bool = df_bool.na.fill(False, subset=['address'])
    display(df_bool)
    myarr = [data[0] for data in df_bool.select('address').collect()]
    print(f'Address columns condition check:- {myarr}')
    signal = np.all(myarr)
    print(f'Signal to keep File:- {signal}')
    if signal == True and first_time == True:
        df_main = df
        first_time = False
    elif signal == True and first_time != True:
        df_main = df_main.union(df)


File Path :- dbfs:/FileStore/demo-files/sample1.csv


name,address
ravi,true
suraj,true
sumit,true


Address columns condition check:- [True, True, True]
Signal to keep File:- True

File Path :- dbfs:/FileStore/demo-files/sample2.csv


name,address
ravi,false
suraj,false
sumit,false
anuj,false


Address columns condition check:- [False, False, False, False]
Signal to keep File:- False

File Path :- dbfs:/FileStore/demo-files/sample3.csv


name,address
ravi,true
suraj,true
sumit,true
vijay,false
james,true


Address columns condition check:- [True, True, True, False, True]
Signal to keep File:- False

File Path :- dbfs:/FileStore/demo-files/sample4.csv


name,address
roy,true
sam,true
Danny,true


Address columns condition check:- [True, True, True]
Signal to keep File:- True


In [0]:
display(df_main)

name,address
ravi,Pune#
suraj,Pune?
sumit,Pune*
roy,kochi#
sam,kohima?
Danny,Mubai*
